# Object-Oriented Design for Coding Interviews (Python)

Practical, interview-focused OOP patterns with clear, runnable examples. Built for LeetCode-style "Design" problems and real-world modeling. Includes step-by-step explanations, trade-offs, and quick-check asserts so you can verify behavior as you learn.

---

## 🧭 Table of Contents

- Overview and Goals
- Core OOP Concepts (Python-specific)
- Classes and Objects (attributes, `__init__`)
- Encapsulation with `@property`
- Class vs Instance; `@classmethod` vs `@staticmethod`
- Inheritance vs Composition (prefer composition)
- Polymorphism & Duck Typing; `typing.Protocol`
- Abstraction with `abc.ABC`
- Data classes (`dataclasses`): `frozen`, `order`, `slots`
- Dunder Methods: `__repr__`, `__eq__`, ordering, iteration
- Context Managers and the Iterator Protocol
- Copying & Mutability: shallow vs deep copy
- Design Patterns (Strategy, Factory, Observer, Adapter)
- LeetCode Design Tie-Ins: RandomizedSet, LRU Cache, Parking System, Hit Counter
- Common Pitfalls + Best Practices
- Exercises
- Summary

---


### Approach to the problem

**Answer these:**

1. **What are two pieces of data the RandomizedSet needs to store to work in $O(1)$?**
    - A **list** to store the actual values (so you can get a random index fast)
    - A **dict** that maps each value → its index in the list (to help you delete in $O(1)$)
      >⚠️ Big Idea:
    Dict gives you fast lookup for presence + index,
    List gives you fast random access for getRandom()

2. **Which Python data structures will you use for those?**
    - **dict** → `val → index in list`
    - Use **list** → store the values
> **Why not just a set?**

>Because sets don’t allow indexed access, so you can’t do random.choice(set) in O(1) without turning it into a list first (which costs $O(n)$). That’s the bottleneck.
So we keep a list ourselves.

3. **Write out in words what should happen in `insert(val)`**
**Here’s the full behavior in words:**

* If `val` is already in the dict, return `False`

*Else:*

* Add val to the end of the list

* Record its index in the dict as `dict[val] = len(list) - 1`

**Dict helps us check presence and store index in the list.**

4. **Write out in words what should happen in `remove(val)`**
The hard part of remove(val) is:

    We can’t remove something from the middle of a list in $O(1)$
    → So we swap it with the last item, pop the last one, and fix the dict.

    Say we want to remove `val = 2`:

    1. Check if `val` is in the dict

    2. If it is:

        * Get its index in the list

        * Swap val with the last element in the list

        * Update the index of that last element in the dict

        * Pop the last element from the list

        * Delete val from the dict

        * Return True

    3. If not present, return `False`

**Think of this like a **“surgical swap-and-pop”** to make removal $O(1)$.**

5. Why is `getRandom()` hard without an array?
* To pick a random item in $O(1)$, you **need to access by index**, like `random.choice()`

* But **sets and dicts don’t support random access**
  #### That’s the problem.

* You’d have to convert them to a list every time → $O(n)$ — too slow

#### So this question is critical because:
> It forces you to choose a list in your internal data model.
Without that, your `getRandom()` won’t meet the time complexity.

## 📈 Level-Up Summary: RandomizedSet Design

| Operation | What Makes It $O(1)$                            | Data Structure     |
|-----------|-------------------------------------------------|--------------------|
| `insert`  | Dict check for existence + append to list       | `dict + list`      |
| `remove`  | Dict lookup for index + swap with end + pop     | `dict + list`      |
| `getRandom` | Random index access from list (`random.choice`) | `list` (direct access) |

###  ☢️ Rule: When to use self.
* Always use `self.` when you're accessing or modifying:
Anything that belongs to the object (shared memory)

That includes:

   * lists like `self.lst`

   * dicts like `self.d`

   * counters, flags, cached values, etc.

**⚠️ self. ties the variable to the object's identity and state**

**‼︎ Without self., you're just working with a temporary local variable**

** 🔌`self.x`, x is accessible from any other method inside that class




In [1]:
import random
val = [[], [1], [2], [2], [], [1], [2], []]
class RandomizedSet:

    def __init__(self):
        self.lst = []
        self.d = {}


    def insert(self, val: int) -> bool:
        if val in self.d:
            return False
        else:
            self.d[val] = len(self.lst)
            self.lst.append(val)
            return True



    def remove(self, val: int) -> bool:
        if val not in self.d:
            return False
        idx = self.d[val]
        last = self.lst[-1]
        self.lst[idx] = last
        self.d[last] = idx
        self.lst.pop()
        del self.d[val]
        return True




    def getRandom(self) -> int:
        return random.choice(self.lst)







### I screwed up the `remove` part
| Step | Action | Code Lick |
|------|--------|-----------|
| 1 | Check presence | `if val not in self.d: return False` |
| 2 | Get index | `idx = self.d[val]` |
| 3 | Get last element | `last = self.lst[-1]` |
| 4 | Overwrite index | `self.lst[idx] = last` |
| 5 | Update dict | `self.d[last] = idx` |
| 6 | Pop last element | `self.lst.pop()` |
| 7 | Remove val from dict | `del self.d[val]` |
| 8 | Return | `return True` |

## 🔍 Understanding the `remove()` Swap Logic in RandomizedSet

---

### 🎯 What is `last = self.lst[-1]` doing?

That line means:

> “Yo, give me the value that’s at the end of the list and name it `last`.”

**Example:**

```
self.lst = [3, 5, 9]
val = 5
self.d = {3: 0, 5: 1, 9: 2}
```
Now you wanna run:
`remove(5)`

Step-by-step:
```
idx = self.d[5]     # idx = 1
last = self.lst[-1] # last = 9
```
You're not changing the list yet — just grabbing the value at the end and storing it in last.
## 🔄 What is `self.lst[idx] = last` doing?

This is the **critical swap step**.

> “Yo, put that last value (`9`) in the position where `val` (`5`) was.”

You're saying:

> "Replace the index of the value I want to remove, with the last value in the list."

---

### 🧪 Example:

```
self.lst = [3, 5, 9]  # Original list
val = 5
self.d = {3: 0, 5: 1, 9: 2}
```

You want to remove 5, which is at index 1.

So:
```
idx = self.d[5]       # idx = 1
last = self.lst[-1]   # last = 9
self.lst[idx] = last  # self.lst[1] = 9
```

Now the list becomes:
```
self.lst = [3, 9, 9]  # ← temporarily duplicates 9
```

Then you pop the last `9`:
```
self.lst.pop()
```

Now the list becomes:
`self.lst = [3, 9]`

### And self.d[last] = idx?
You now gotta update the dict to say:

> "Yo, 9 moved — it’s no longer at index 2, it’s now at index 1."

So you do:
`self.d[9] = 1`

This is crucial.

If you don’t do this step, `self.d[9]` will still say 9 lives at index 2 — which no longer exists in the list.

💣 That’s a ticking bug waiting to happen.




# 🧠 OOP + Advanced Hashing: Mastering `RandomizedSet` (LeetCode 380)

---

## 🔍 Problem Summary

**Design a class with:**
- `insert(val)` → Insert value if not present, return True/False
- `remove(val)` → Remove value if present, return True/False
- `getRandom()` → Return a random element from the set

**All operations must run in average $O(1)$ time.**

---

## 🧱 Internal Data Structures

| Name        | Purpose                           | Type        |
|-------------|------------------------------------|-------------|
| `self.lst`  | Stores actual values (for random) | `list`      |
| `self.d`    | Maps value → index in list        | `dict`      |

This combo lets us:
- Append values efficiently
- Access random values via index
- Delete in $O(1)$ via swap-then-pop

---

## 🧠 OOP Mental Rules

- Use `self.var` for anything you want the **object to remember**
- Use local variables (`val`, `idx`, etc.) for one-time logic
- Only use `self.` when you need the variable to persist between method calls

---

## 🚀 Method Logic Overview

### `insert(val)`

Steps:
- Check presence: `if val in self.d: return False`
- Record index: `self.d[val] = len(self.lst)`
- Append to list: `self.lst.append(val)`
- Return `True`


In [2]:
# Insert logic demo (standalone snippet)
_demo_lst = []
_demo_d = {}
val = 42
if val in _demo_d:
    inserted = False
else:
    _demo_d[val] = len(_demo_lst)
    _demo_lst.append(val)
    inserted = True
print(inserted, _demo_lst, _demo_d)


True [42] {42: 0}


**remove(val)**
```
if val not in self.d:
    return False
idx = self.d[val]
last = self.lst[-1]
self.lst[idx] = last
self.d[last] = idx
self.lst.pop()
del self.d[val]
return True
```

## 🔁 Swap-to-Pop Pattern

| Line                  | Meaning                                                   |
|-----------------------|-----------------------------------------------------------|
| `last = self.lst[-1]` | Get the last element in the list                          |
| `self.lst[idx] = last`| Overwrite the removed val's spot with last's value        |
| `self.d[last] = idx`  | Update the dict to reflect last’s new index               |
| `self.lst.pop()`      | Remove the now-duplicate last element from the list       |
| `del self.d[val]`     | Remove the original value from the dict                   |


**getRandom()**
```
return random.choice(self.lst)
```

* Uniformly selects a value from the list

* List allows $O(1)$ access

* Don't forget: import random at the top

## ✅ Lessons Burned In

- `self.` ties a variable to **object state**
- `dict + list` = powerful tool for **$O(1)$** operations
- Never remove from the middle of a list — **swap with last, then pop**
- Design each method with **efficiency** and **state management** in mind
- **Return values** (`True` / `False`) matter in method-based problems





In [3]:
import random

class RandomizedSet:
    """RandomizedSet with average O(1) insert, remove, and getRandom.

    Uses two structures:
      - self.lst: list[int] for O(1) random index access
      - self.d: dict[int, int] mapping value -> index in self.lst for O(1) lookup/updates

    Average-time complexity:
      - insert:   O(1)
      - remove:   O(1) via swap-with-last + pop
      - getRandom: O(1)
    """

    def __init__(self):
        # Store current values here; enables O(1) random access by index
        self.lst: list[int] = []
        # Map each value to its index within self.lst for constant-time checks/updates
        self.d: dict[int, int] = {}

    def insert(self, val: int) -> bool:
        """Insert value if absent.

        Returns:
            True if the value was added; False if it already existed.
        """
        # Presence check is O(1) using the dict
        if val in self.d:
            return False
        # Record the index where val will be placed (end of list)
        self.d[val] = len(self.lst)
        # Append keeps amortized O(1)
        self.lst.append(val)
        return True

    def remove(self, val: int) -> bool:
        """Remove value if present.

        Key idea: swap the element-to-remove with the last element, then pop.
        This avoids O(n) deletions from the middle of a list and keeps it O(1).
        Returns True if removed; False if not found.
        """
        if val not in self.d:
            return False
        # Index of the element to delete
        idx = self.d[val]
        # Value at the end of the list (will move to idx)
        last = self.lst[-1]
        # Overwrite the target slot with the last element
        self.lst[idx] = last
        # Update the moved element's index in the dict
        self.d[last] = idx
        # Remove the duplicate last slot from the list
        self.lst.pop()
        # Finally, remove the value from the dict
        del self.d[val]
        return True

    def getRandom(self) -> int:
        """Return a uniformly random element from the set in O(1)."""
        # random.choice is O(1) on lists because it uses an index
        return random.choice(self.lst)


# Simulating LeetCode-style input
# This mimics how LeetCode feeds operations and their arguments to your class.
# ops:  a list of operation names as strings
# vals: a parallel list where each element is the argument list for the op at the same index
ops = ["RandomizedSet", "insert", "remove", "insert", "getRandom", "remove", "insert", "getRandom"]
vals = [[],              [1],       [2],       [2],       [],          [1],       [2],       []]

# We'll capture each operation's return value in `res` to verify behavior.
res: list = []
obj: RandomizedSet | None = None

for op, args in zip(ops, vals):
    if op == "RandomizedSet":
        # Constructor call: create a new instance; LeetCode expects `None` as the recorded result
        obj = RandomizedSet()
        res.append(None)
    elif op == "insert":
        # Insert the provided value (args is a one-element list like [x])
        res.append(obj.insert(args[0]))  # True if inserted, False if duplicate
    elif op == "remove":
        # Remove the provided value if present
        res.append(obj.remove(args[0]))  # True if removed, False if not found
    elif op == "getRandom":
        # Return a uniformly random element from the set
        res.append(obj.getRandom())

print(res)  # Example output pattern mirrors LeetCode's expected format




[None, True, False, True, 1, True, False, 2]


In [4]:
import random

class RandomizedSet:
    def __init__(self):
        self.lst = []
        self.d = {}

    def insert(self, val: int) -> bool:
        if val in self.d:
            return False
        self.d[val] = len(self.lst)
        self.lst.append(val)
        return True

    def remove(self, val: int) -> bool:
        if val not in self.d:
            return False
        idx = self.d[val]
        last = self.lst[-1]
        self.lst[idx] = last
        self.d[last] = idx
        self.lst.pop()
        del self.d[val]
        return True

    def getRandom(self) -> int:
        return random.choice(self.lst)


# ✅ Manual test
obj = RandomizedSet()
print(obj.insert(5))      # True
print(obj.remove(3))      # False
print(obj.insert(6))      # True
print(obj.getRandom())    # 1 or 2
print(obj.remove(1))      # True
print(obj.insert(2))      # False (2 already in set)
print(obj.getRandom())    # 2


True
False
True
6
False
True
5


In [9]:
s = "A man, a plan, a canal: Panama"

def Palin(s):
    res = ''
    for char in s:
        if char.isalnum():
            res += char.lower()
        return res == res[::-1]
    return False

Palin(s)

True

## Mini OOP Starter Toolbox


In [10]:
class MyThing:
    def __init__(self):
        self.lst = []
        self.d = {}
        self.set = set()
        self.counter = 0

    def add(self, val):
        if val not in self.set:
            self.set.add(val)
            self.lst.append(val)

    def remove(self, val):
        if val in self.set:
            self.set.remove(val)
            # remove from list, dict if needed

    def get_random(self):
        return random.choice(self.lst)


## OOP Problem

#### Problem: Design a MinStack
**Design a stack that supports all the following operations in $O(1)$ time:**

`push(val)` — Pushes the element val onto the stack.

`pop()` — Removes the element on the top of the stack.

`top()` — Gets the top element.

`getMin()` — Retrieves the minimum element in the stack.

You must implement the functions of the class such that each function works in constant time.

In [11]:
class MinStack:

    def __init__(self):
        self.stack = []
        self.min_stack = [] # need to create a separate min_stack so it persists between method calls keeps it O(1)

    def push(self, val: int) -> None:
        self.stack.append(val)
        if not self.min_stack:
            self.min_stack.append(val)
        else:
            self.min_stack.append(min(val, self.min_stack[-1])) # new min-so-far


    def pop(self) -> None:
        self.stack.pop()
        self.min_stack.pop()

    def top(self) -> int:
        return self.stack[-1] # In Python, list[-1] is the top of your stack.pop() removes it, [-1] peeks at it

    def getMin(self) -> int:
        return self.min_stack[-1]
